In [1]:
# --- Core Libraries ---
import os
import random
import json
import pandas as pd
import numpy as np
import torch
import time
from tqdm import tqdm
import re

# --- Hugging Face: Dataset, Tokenizer, Model ---
from datasets import load_dataset, DatasetDict, Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    TrainingArguments, 
    Trainer, 
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    pipeline
)

# --- LoRA & Parameter-Efficient Tuning ---
from peft import LoraConfig, get_peft_model, TaskType, PeftModel

# --- W&B Experiment Tracking ---
import wandb

# --- SQL Evaluation ---
import sqlite3
import sqlparse
from tabulate import tabulate
import evaluate  # for BLEU, ROUGE
import nltk
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sidpk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sidpk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
os.environ["WANDB_NOTEBOOK_NAME"] = "text2sql_finetune_and_eval.ipynb"

In [3]:
torch.cuda.empty_cache()

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    print("GPU not detected — will fall back to CPU.")

PyTorch version: 2.5.1+cu121
CUDA available: True
Using GPU: NVIDIA GeForce RTX 4050 Laptop GPU


In [4]:
# Load dataset
dataset = load_dataset("Clinton/Text-to-SQL-v1")
shuffled_dataset = dataset.shuffle(seed=42)

df = pd.DataFrame(shuffled_dataset["train"])
df.sample(5)

,instruction,input,response,source,text
119876,What was the result of the game that was atten...,CREATE TABLE table_name_47 (\n result VARCH...,SELECT result FROM table_name_47 WHERE attenda...,sql_create_context,Below are sql tables schemas paired with instr...
246771,"What is the low money with 374,164$ of debt an...","CREATE TABLE table_54884 (\n ""Candidate"" te...","SELECT MIN(""Money Spent"") FROM table_54884 WHE...",wikisql,Below are sql tables schemas paired with instr...
82016,What was the Result on 11/02/1946?,CREATE TABLE table_name_84 (\n result VARCH...,SELECT result FROM table_name_84 WHERE date = ...,sql_create_context,Below are sql tables schemas paired with instr...
134146,What is the report for round 9?,CREATE TABLE table_name_9 (\n report VARCHA...,SELECT report FROM table_name_9 WHERE round = 9,sql_create_context,Below are sql tables schemas paired with instr...
28258,For those employees who do not work in departm...,CREATE TABLE job_history (\n EMPLOYEE_ID de...,"SELECT HIRE_DATE, SUM(SALARY) FROM employees W...",nvbench,Below are sql tables schemas paired with instr...


In [5]:
print("Any nulls?", df.isna().sum())
print("Any empty strings?", (df == "").sum())
print("Unique columns:", df.columns)

Any nulls? instruction    0
input          0
response       0
source         0
text           0
dtype: int64
Any empty strings? instruction    2
input          0
response       0
source         0
text           0
dtype: int64
Unique columns: Index(['instruction', 'input', 'response', 'source', 'text'], dtype='object')


In [6]:
df_clean = df[df["instruction"] != ""].reset_index(drop=True)
print(f"Filtered dataset size: {len(df_clean)}")

Filtered dataset size: 262206


In [7]:
formatted_dataset = Dataset.from_pandas(df_clean[["text"]])
formatted_dataset = formatted_dataset.train_test_split(test_size=0.1, seed=42)

print(formatted_dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 235985
    })
    test: Dataset({
        features: ['text'],
        num_rows: 26221
    })
})


In [8]:
# Load Tokenizer

model_name = "deepseek-ai/deepseek-coder-1.3b-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [9]:
# Find max length of instructions to pick the optimal max prompt length

# Function to compute token length stats
def compute_token_stats(dataset_split, tokenizer):
    lengths = [len(tokenizer(x)["input_ids"]) for x in dataset_split["text"]]
    stats = {
        "max": int(np.max(lengths)),
        "95th_percentile": int(np.percentile(lengths, 95)),
        "mean": round(np.mean(lengths), 2),
        "min": int(np.min(lengths)),
        "num_samples": len(lengths),
    }
    return stats

# Compute for both splits
train_stats = compute_token_stats(formatted_dataset["train"], tokenizer)
test_stats = compute_token_stats(formatted_dataset["test"], tokenizer)

print("Train Token Length Stats:", train_stats)
print("Test Token Length Stats:", test_stats)

Train Token Length Stats: {'max': 3226, '95th_percentile': 1432, 'mean': np.float64(376.65), 'min': 66, 'num_samples': 235985}
Test Token Length Stats: {'max': 3220, '95th_percentile': 1447, 'mean': np.float64(380.84), 'min': 68, 'num_samples': 26221}


In [10]:
#looking at the max token size in the entire data response
sql_token_lengths = df_clean["response"].apply(lambda x: len(tokenizer(x, truncation=False)["input_ids"]))

# Analyze
print("Mean SQL Response token length:", sql_token_lengths.mean())
print("95th percentile:", sql_token_lengths.quantile(0.95))
print("Max SQL Response token length:", sql_token_lengths.max())

Mean SQL Response token length: 51.61714834900803
95th percentile: 162.0
Max SQL Response token length: 1868


In [11]:
#Smart Padding
def tokenize(examples):
    input_ids_list = []
    attention_mask_list = []
    labels_list = []
    
    max_length = 4096

    for full_text in examples["text"]:
        # Extract prompt and response
        prompt_text = full_text.split("### Response:")[0].strip() + "\n### Response:\n"
        response_text = full_text.split("### Response:")[1].strip()
        
        # Tokenize with truncation
        prompt_tokens = tokenizer(prompt_text, truncation=True, max_length=max_length)["input_ids"]
        response_tokens = tokenizer(response_text, truncation=True, max_length=max_length)["input_ids"]
        response_tokens.append(tokenizer.eos_token_id)
        
        # Combine tokens for input
        input_ids = prompt_tokens + response_tokens
        attention_mask = [1] * len(input_ids)
        
        # Create labels - keep prompt tokens, mask response tokens
        labels = input_ids.copy()  # Start with full sequence
        labels = [-100] * len(prompt_tokens) + response_tokens #mask prompt tokens

        input_ids_list.append(input_ids)
        attention_mask_list.append(attention_mask)
        labels_list.append(labels)

    return {
        "input_ids": input_ids_list,
        "attention_mask": attention_mask_list,
        "labels": labels_list
    }

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # because this is causal LM
    pad_to_multiple_of=16  # speeds up training on GPU
)

In [12]:
import sqlite3
import re

def fix_missing_semicolons(sql_code):
    """
    Inserts semicolons between multiple CREATE TABLE statements if missing.
    Looks for patterns like `) CREATE TABLE` and adds a semicolon between them.
    """
    return re.sub(r'\)\s*(?=CREATE TABLE)', r');\n', sql_code.strip())

def can_execute_sql(generated_sql, schema=None, verbose=True):
    """
    Check if a SQL query or script can be executed against a given schema.

    Args:
        generated_sql (str): The SQL query or script to test.
        schema (str, optional): The database schema to create before testing.
        verbose (bool, optional): Whether to print detailed errors.

    Returns:
        tuple: (bool, str) - (success status, message or error)
    """
    conn = None
    try:
        conn = sqlite3.connect(":memory:")
        cursor = conn.cursor()

        # Create schema if provided
        if schema:
            try:
                schema = fix_missing_semicolons(schema)
                cursor.executescript(schema)
                conn.commit()
            except sqlite3.Error as e:
                if verbose:
                    print("Schema execution failed.")
                    print("Error:", e)
                return False

        # Execute the query or script
        try:
            if ';' in generated_sql.strip().rstrip(';'):
                cursor.executescript(generated_sql)
                return True
            else:
                cursor.execute(generated_sql)
                return True
        except sqlite3.Error as e:
            if verbose:
                print("Query execution failed.")
                print("Error:", e)
            return False

    except Exception as e:
        if verbose:
            print("General error.")
            print("Error:", e)
        return False

    finally:
        if conn:
            conn.close()


In [13]:
#computing the metrics for the baseline model based on similarilty of output, sql compilation and time

# Load metrics
meteor_metric = evaluate.load("meteor")

def extract_sql_from_output(output_text, prompt_text):
    """Extract SQL query from model output, handling various formats."""
    # Remove the prompt from the output
    sql_text = output_text[len(prompt_text):].strip()
    
    # Remove any markdown code blocks if present
    sql_text = re.sub(r'```sql\s*|\s*```', '', sql_text)
    sql_text = re.sub(r'```\s*|\s*```', '', sql_text)
    
    # Remove any trailing text after semicolon
    if ';' in sql_text:
        sql_text = sql_text.split(';')[0] + ';'
    
    return sql_text.strip()

def evaluate_model_on_dataset(
    model,
    tokenizer,
    dataset,
    max_new_tokens=2048
):
    predictions = []
    references = []
    compile_success = 0
    execution_times = []

    dataset_slice = dataset

    for example in tqdm(dataset_slice, desc="Evaluating"):
        # Extract prompt and response using the same format as tokenize function
        prompt_text = example["text"].split("### Response:")[0].strip() + "\n### Response:\n"
        ground_truth = example["text"].split("### Response:")[1].strip()
        schema = example["text"].split("### Input:")[1].split("### Response:")[0].strip()

        inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                eos_token_id=tokenizer.eos_token_id,
                max_new_tokens=2048,
                pad_token_id=tokenizer.eos_token_id
                )
        
        # Get the generated SQL - everything after the prompt
        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_sql = extract_sql_from_output(decoded, prompt_text)
        print("SQL Output:", generated_sql)

        # Add prediction for METEOR
        predictions.append(generated_sql)
        references.append([ground_truth])  # METEOR expects references as a list of lists

        # Compile SQL Query and measure time
        start_time = time.perf_counter()
        success = can_execute_sql(generated_sql, schema)
        end_time = time.perf_counter()

        if success:
            compile_success += 1
            execution_times.append(end_time - start_time)

    # Compute metrics
    meteor_score = meteor_metric.compute(predictions=predictions, references=references)["meteor"]
    sql_compilation_rate = compile_success / len(dataset_slice)
    
    # Calculate average execution time for successful queries
    avg_execution_time = sum(execution_times) / len(execution_times) if execution_times else 0

    metrics = {
        "meteor_score": round(meteor_score, 4),
        "sql_compilation_rate": round(sql_compilation_rate, 4),
        "avg_execution_time_ms": round(avg_execution_time * 1000, 2),  # Convert to milliseconds
        "num_eval_samples": len(dataset_slice),
        "num_successful_queries": compile_success
    }

    return metrics

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sidpk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\sidpk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sidpk\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [14]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

In [15]:
#starting the finetuning process

In [16]:
torch.cuda.empty_cache()

In [17]:
wandb.init(
    project="deepseek-sql-finetune",
    name="baseline-run",
    notes="1.3B model with QLoRA, loss tracking"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sidpkul21 (sidpkul21-georgia-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [18]:
training_args = TrainingArguments(
    output_dir="./deepseek-coder-qlora-sql-25k",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=1e-5,
    warmup_ratio=0.05,
    lr_scheduler_type="cosine",
    num_train_epochs=3,
    logging_steps=100,
    save_steps=1200,
    fp16=True,
    report_to="wandb",
    run_name="deepseek-coder-qlora-sql-25k",
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=1200,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=10,
    max_grad_norm=1.0,
    label_names=["labels"] 
)

In [19]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # or ["query_key_value"] depending on model architecture
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model_finetune = get_peft_model(base_model, lora_config)

In [20]:
tokenized_dataset = formatted_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/235985 [00:00<?, ? examples/s]

Map:   0%|          | 0/26221 [00:00<?, ? examples/s]

In [21]:
small_train = tokenized_dataset["train"].select(range(25000))
small_eval = tokenized_dataset["test"].select(range(2500))

trainer = Trainer(
    model=model_finetune,
    args=training_args,
    train_dataset=small_train, 
    eval_dataset=small_eval,
    tokenizer=tokenizer,
    data_collator=data_collator
)

C:\Users\sidpk\AppData\Local\Temp\ipykernel_17688\2084438354.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
trainer.train()

Step,Training Loss,Validation Loss
1200,0.599400,0.681762
2400,0.474200,0.612577
3600,0.419400,0.584481
4800,0.392600,0.571407
6000,0.413500,0.563833
7200,0.373700,0.560163
8400,0.409400,0.558614


TrainOutput(global_step=9375, training_loss=0.4877398893229167, metrics={'train_runtime': 28033.307, 'train_samples_per_second': 2.675, 'train_steps_per_second': 0.334, 'total_flos': 2.2400043647945933e+17, 'train_loss': 0.4877398893229167, 'epoch': 3.0})